# DX 704 Week 8 Project

This homework will modify a simulator controlling a small vehicle to implement tabular q-learning.
You will first test your code with random and greedy-epsilon policies, then tweak your own training method for a more optimal policy.

The full project description and a template notebook are available on GitHub: [Project 8 Materials](https://github.com/bu-cds-dx704/dx704-project-08).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Rover Simulator

The following Python class implements a simulation of a simple vehicle with integer x,y coordinates facing in one of 8 possible directions.


In [1]:
# DO NOT CHANGE

import random

class RoverSimulator(object):
    DIRECTIONS = ((0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0), (-1, 1))

    def __init__(self, resolution):
        self.resolution = resolution
        self.terminal_state = self.construct_state(resolution // 2, resolution // 2, 0)

        self.initial_states = []
        for initial_x in (0, resolution // 2, resolution - 1):
            for initial_y in (0, resolution // 2, resolution - 1):
                for initial_direction in range(8):
                    initial_state = self.construct_state(initial_x, initial_y, initial_direction)
                    if initial_state != self.terminal_state:
                        self.initial_states.append(initial_state)

    def construct_state(self, x, y, direction):
        assert 0 <= x < self.resolution
        assert 0 <= y < self.resolution
        assert 0 <= direction < 8

        state = (y * self.resolution + x) * 8 + direction
        assert self.decode_state(state) == (x, y, direction)
        return state

    def decode_state(self, state):
        direction = state % 8
        x = (state // 8) % self.resolution
        y = state // (8 * self.resolution)

        return (x, y, direction)

    def get_actions(self, state):
        return [-1, 0, 1]

    def get_next_reward_state(self, curr_state, curr_action):
        if curr_state == self.terminal_state:
            # no rewards or changes from terminal state
            return (0, curr_state)

        (curr_x, curr_y, curr_direction) = self.decode_state(curr_state)
        (curr_dx, curr_dy) = self.DIRECTIONS[curr_direction]

        assert self.construct_state(curr_x, curr_y, curr_direction) == curr_state

        assert curr_action in (-1, 0, 1)

        next_x = min(max(0, curr_x + curr_dx), self.resolution - 1)
        next_y = min(max(0, curr_y + curr_dy), self.resolution - 1)
        next_direction = (curr_direction + curr_action) % 8

        next_state = self.construct_state(next_x, next_y, next_direction)
        next_reward = 1 if next_state == self.terminal_state else 0

        return (next_reward, next_state)

    def rollout_policy(self, policy_func, max_steps=1000):
        curr_state = self.sample_initial_state()
        for i in range(max_steps):
            curr_action = policy_func(curr_state, self.get_actions(curr_state))
            (next_reward, next_state) = self.get_next_reward_state(curr_state, curr_action)
            yield (curr_state, curr_action, next_reward, next_state)
            curr_state = next_state

    def sample_initial_state(self):
        return random.choice(self.initial_states)

In [2]:
simulator = RoverSimulator(16)
initial_sample = simulator.sample_initial_state()
print("INITIAL SAMPLE", initial_sample)

INITIAL SAMPLE 1029


## Part 1: Implement a Random Policy

Random policies are often used to test simulators and start initial exploration.
Implement a random policy for these simulators.

In [3]:
# YOUR CHANGES HERE

def random_policy(state, actions):
    return 0

In [4]:
import csv, random

# --- Part 1: random policy
def random_policy(state, actions):
    return random.choice(actions)

# run once and SAVE: log-random.tsv
with open("log-random.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["curr_state","curr_action","next_reward","next_state"])
    for (cs, ca, nr, ns) in simulator.rollout_policy(random_policy, max_steps=32):
        w.writerow([cs, ca, nr, ns])

print("Wrote log-random.tsv")


Wrote log-random.tsv


Use the code below to test your random policy.
Then modify it to save the results in "log-random.tsv" with the columns curr_state, curr_action, next_reward and next_state.

In [5]:
# YOUR CHANGES HERE

for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=32):
    print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

CURR STATE 1091 ACTION 1 NEXT REWARD 0 NEXT STATE 972
CURR STATE 972 ACTION 1 NEXT REWARD 0 NEXT STATE 845
CURR STATE 845 ACTION -1 NEXT REWARD 0 NEXT STATE 708
CURR STATE 708 ACTION 1 NEXT REWARD 0 NEXT STATE 581
CURR STATE 581 ACTION -1 NEXT REWARD 0 NEXT STATE 444
CURR STATE 444 ACTION -1 NEXT REWARD 0 NEXT STATE 315
CURR STATE 315 ACTION -1 NEXT REWARD 0 NEXT STATE 194
CURR STATE 194 ACTION 1 NEXT REWARD 0 NEXT STATE 203
CURR STATE 203 ACTION 1 NEXT REWARD 0 NEXT STATE 84
CURR STATE 84 ACTION 1 NEXT REWARD 0 NEXT STATE 85
CURR STATE 85 ACTION 0 NEXT REWARD 0 NEXT STATE 77
CURR STATE 77 ACTION 1 NEXT REWARD 0 NEXT STATE 70
CURR STATE 70 ACTION -1 NEXT REWARD 0 NEXT STATE 61
CURR STATE 61 ACTION -1 NEXT REWARD 0 NEXT STATE 52
CURR STATE 52 ACTION 0 NEXT REWARD 0 NEXT STATE 52
CURR STATE 52 ACTION -1 NEXT REWARD 0 NEXT STATE 51
CURR STATE 51 ACTION 1 NEXT REWARD 0 NEXT STATE 60
CURR STATE 60 ACTION -1 NEXT REWARD 0 NEXT STATE 59
CURR STATE 59 ACTION -1 NEXT REWARD 0 NEXT STATE 66
CURR

Submit "log-random.tsv" in Gradescope.

## Part 2: Implement Q-Learning with Random Policy

The code below runs 32 random rollouts of 1024 steps using your random policy.
Modify the rollout code to implement Q-Learning.
Just implement one learning update for each sampled state-action in the simulation.
Use $\alpha=1$ and $\gamma=0.9$ since the simulator is deterministic and there is a sink where the rewards stop.




In [6]:
# YOUR CHANGES HERE

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(random_policy, max_steps=1024):
        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)
        pass

Save each step in the simulator in a file "q-random.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

In [7]:
# YOUR CHANGES HERE

# --- Part 2: tabular Q-learning with random policy
from collections import defaultdict

gamma = 0.9
alpha = 1.0   # deterministic env

Q = defaultdict(float)

def q_get(s, a): 
    return Q[(s,a)]

def q_set(s, a, v): 
    Q[(s,a)] = v

def max_q_next(s_next):
    acts = simulator.get_actions(s_next)
    return max(q_get(s_next,a) for a in acts)

# learn + LOG every update
with open("q-random.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["curr_state","curr_action","next_reward","next_state","old_value","new_value"])
    for episode in range(32):
        for (cs, ca, nr, ns) in simulator.rollout_policy(random_policy, max_steps=1024):
            old = q_get(cs, ca)
            target = nr + gamma * max_q_next(ns)
            new = target if alpha == 1.0 else (1 - alpha)*old + alpha*target
            q_set(cs, ca, new)
            w.writerow([cs, ca, nr, ns, old, new])

print("Wrote q-random.tsv")


Wrote q-random.tsv


Submit "q-random.tsv" in Gradescope.

## Part 3: Implement Epsilon-Greedy Policy

Implement an epsilon-greedy policy that picks the optimal policy based on your q-values so far 75% of the time, and picks a random action 25% of the time.
This is a high epsilon value, but the environment is deterministic, so it will benefit from more exploration.

In [8]:
# YOUR CHANGES HERE

# hard-code epsilon=0.25. this is high but the environment is deterministic.
def epsilon_greedy_policy(state, actions):
    return 0

Combine your epsilon-greedy policy with q-learning below and save the observations and updates in "q-greedy.tsv" with columns curr_state, curr_action, next_reward, next_state, old_value, new_value.

Hint: make sure to reset your q-learning state before running the simulation below so that the learning process is recorded from the beginning.

In [9]:
# YOUR CHANGES HERE

for episode in range(32):
    for (curr_state, curr_action, next_reward, next_state) in simulator.rollout_policy(epsilon_greedy_policy, max_steps=1024):
        #print("CURR STATE", curr_state, "ACTION", curr_action, "NEXT REWARD", next_reward, "NEXT STATE", next_state)

        if next_reward > 0:
            # moving to terminal state
            break

        pass

In [10]:
# --- Part 3: reset Q and use epsilon-greedy policy
Q = defaultdict(float)
epsilon = 0.25
gamma = 0.9
alpha = 1.0

def greedy_action(state):
    actions = simulator.get_actions(state)
    # tie-break by fixed order to keep deterministic
    best = max(actions, key=lambda a: Q[(state,a)])
    return best

def epsilon_greedy_policy(state, actions):
    if random.random() < epsilon:
        return random.choice(actions)
    return greedy_action(state)

with open("q-greedy.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["curr_state","curr_action","next_reward","next_state","old_value","new_value"])
    for episode in range(32):
        # fresh rollout from a random initial state
        for (cs, _, _, _) in simulator.rollout_policy(lambda s, a: 0, max_steps=0):
            pass  # (no-op to satisfy structure)
        # do our own loop to access the chosen action
        curr_state = simulator.sample_initial_state()
        for step in range(1024):
            ca = epsilon_greedy_policy(curr_state, simulator.get_actions(curr_state))
            nr, ns = simulator.get_next_reward_state(curr_state, ca)

            old = Q[(curr_state, ca)]
            target = nr + gamma * max(Q[(ns,a)] for a in simulator.get_actions(ns))
            new = target if alpha == 1.0 else (1 - alpha)*old + alpha*target
            Q[(curr_state, ca)] = new

            w.writerow([curr_state, ca, nr, ns, old, new])

            curr_state = ns
            if nr > 0:  # reached terminal, stop this episode early
                break

print("Wrote q-greedy.tsv")


Wrote q-greedy.tsv


Submit "q-greedy.tsv" in Gradescope.

## Part 4: Extract Policy from Q-Values

Using your final q-values from the previous simulation, extract a policy picking the best actions according to those q-values.
Save the policy in a file "policy-greedy.tsv" with columns state and action.

In [11]:
# YOUR CHANGES HERE

# --- Part 4: write a policy action for EVERY state in the grid
def all_states(sim):
    states = []
    for y in range(sim.resolution):
        for x in range(sim.resolution):
            for d in range(8):
                states.append(sim.construct_state(x, y, d))
    return states

with open("policy-greedy.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["state","action"])
    for s in all_states(simulator):
        actions = simulator.get_actions(s)
        # default to 0 if Q never saw this state-action
        best = max(actions, key=lambda a: Q[(s,a)])
        w.writerow([s, best])

print("Wrote policy-greedy.tsv")


Wrote policy-greedy.tsv


Submit "policy-greedy.tsv" in Gradescope.

## Part 5: Implement Large Policy

Train a more optimal policy using q-learning.
Save the policy in a file "policy-optimal.tsv" with columns state and action.

Hint: this policy will be graded on its performance compared to optimal for each of the initial states.
**You will get full credit if the average value of your policy for the initial states is within 20% of optimal.**
Make sure that your policy has coverage of all the initial states, and does not take actions leading to states not included in your policy.
You will have to run several rollouts to get coverage of all the initial states, and the provided loops for parts 2 and 3 only consist of one rollout each.

Hint: this environment only gives one non-zero reward per episode, so you may want to cut off rollouts for speed once they get that reward.
But make sure you update the q-values first!

In [12]:
# YOUR CHANGES HERE

# --- Part 5: more training with many short episodes and epsilon decay
Q = defaultdict(float)
gamma = 0.9
alpha = 1.0

def eps_greedy(s, eps):
    if random.random() < eps:
        return random.choice(simulator.get_actions(s))
    return max(simulator.get_actions(s), key=lambda a: Q[(s,a)])

episodes = 6000
max_steps = 256
eps_start, eps_end = 0.6, 0.05

for ep in range(episodes):
    # linear epsilon decay
    eps = eps_start + (eps_end - eps_start) * (ep / max(1, episodes - 1))
    s = simulator.sample_initial_state()
    for t in range(max_steps):
        a = eps_greedy(s, eps)
        r, s2 = simulator.get_next_reward_state(s, a)

        # TD(0) update with α=1 (deterministic)
        Q[(s,a)] = r + gamma * max(Q[(s2,b)] for b in simulator.get_actions(s2))

        s = s2
        if r > 0:
            break  # reached goal; next episode

# export full policy for ALL states (coverage safe for grader)
with open("policy-optimal.tsv", "w", newline="") as f:
    w = csv.writer(f, delimiter="\t")
    w.writerow(["state","action"])
    for s in all_states(simulator):
        actions = simulator.get_actions(s)
        best = max(actions, key=lambda a: Q[(s,a)])
        w.writerow([s, best])

print("Wrote policy-optimal.tsv")


Wrote policy-optimal.tsv


Submit "policy-optimal.tsv" in Gradescope.

## Part 6: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.
You do not need to provide code for data collection if you did that by manually.

## Part 7: Acknowledgements

If you discussed this assignment with anyone, please acknowledge them here.
If you did this assignment completely on your own, simply write none below.

If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for. If you did not use any other libraries, simply write none below.

If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy. If you did not use any generative AI tools, simply write none below.

In [13]:
with open("acknowledgments.txt","w") as f:
    f.write("none\n")
print("Wrote acknowledgments.txt")


Wrote acknowledgments.txt
